參考
https://www.kaggle.com/apapiu/regularized-linear-models

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

data_path = '/data/examples/pm25/'
#E:\AI\pm25

%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

In [2]:
train = pd.read_csv(data_path + "PM25_train.csv")
test = pd.read_csv(data_path + "PM25_test.csv")

print(train.shape, test.shape)

(1116554, 10) (41223, 9)


In [3]:
#remove outliers

def removeOutliers(dataFrame):
    print('old', dataFrame.shape)

    devices = dataFrame['device_id'].unique()

    newDf = pd.DataFrame()

    for device in devices:
        deviceDf = dataFrame[dataFrame['device_id']==device]
        deviceDf = deviceDf[np.abs(deviceDf.PM10-deviceDf.PM10.mean()) <= (3*deviceDf.PM10.std())]
        #print(device,deviceDf.shape)
        newDf = pd.concat([deviceDf, newDf])  

    newDf = newDf.reset_index(drop=True)

    print('new', newDf.shape)
    return newDf


In [ ]:
train = removeOutliers(train)
test = removeOutliers(test)


old (1116554, 10)
new (1112142, 10)
old (41223, 9)


In [ ]:

#y = np.log1p(train['PM2.5'])
y = train['PM2.5']
print(y.shape)
y.head()

In [ ]:
testIds = test['device_id']

train=train.drop(columns=['PM2.5'])
#test.insert(loc=3, column='PM2.5', value=0.0)
#test.head()


In [ ]:
all_data = pd.concat((train.loc[:,],
                      test.loc[:,]))
print(all_data.shape)

In [ ]:

all_data=all_data.drop(columns=['Temperature','Humidity','lat','lon'])
all_data.head()

In [ ]:
all_data.info()

In [ ]:
all_data['Time'] = all_data['Time'].apply(lambda x: x[:2]) #compute skewness

all_data.head()


In [ ]:

for c in all_data.columns:
    if(all_data[c].dtype == np.object):
        all_data[c] = all_data[c].astype('category').cat.codes


In [ ]:
all_data.head()

In [ ]:
all_data.info()

In [ ]:
#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]


In [ ]:
print(X_train.shape,X_test.shape,y.shape)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split


In [ ]:
isXbg = True
if isXbg:        
    dtrain = xgb.DMatrix(X_train, label = y)
    dtest = xgb.DMatrix(X_test)

    params = {"max_depth":2, "eta":0.1}
    #cvresult = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)


    #print('Best number of trees = {}'.format(cvresult.shape[0]))
    #cvresult

    #cvresult.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()
    model = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1) #the params were tuned using xgb.cv
    #model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)
    model.fit(X_train, y, eval_metric='rmse')
else:
    model = LassoCV(alphas = [1, 0.1, 0.001, 0.0005, 0.0007]).fit(X_train, y)
    coef = pd.Series(model.coef_, index = X_train.columns)
    print('best alpha=',model.alpha_,"Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")


In [ ]:


rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 6))
print(rmse, rmse.mean())



In [ ]:
#測試mean_squared_error
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_train)
print('mean_squared_error ',np.sqrt(mean_squared_error(y, y_pred)))

In [ ]:
y_pred

In [ ]:
y_pred = model.predict(X_test)

out = pd.DataFrame()
out['device_id'] = testIds
#out['pred_pm25'] =  np.expm1(y_pred)
out['pred_pm25'] =  y_pred
print(out.shape)


In [ ]:
out = out.groupby(by=['device_id'], as_index=False).mean()


In [ ]:
out.head()

In [ ]:
out.shape

In [ ]:
out.to_csv('pm25_submit.csv',index=False)